In [1]:
from planetaryimage import PDS3Image
from matplotlib import pylab as plt
import numpy as np
import matplotlib.patches as patches
import requests
from bs4 import BeautifulSoup
import os
import zipfile
import io
import alphashape
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import shutil
from shapely.errors import TopologicalError
import math

In [2]:
# Parámetros y configuración inicial
volumenes = [35, 45, 65, 82, 86, 87, 93, 98, 100, 101, 108, 111, 120, 126, 127, 131, 149, 157, 161, 166, 167, 174, 177, 181, 189, 193, 195, 199, 200, 201, 203, 209, 210, 211, 218, 220, 229, 234, 239, 240, 243, 248, 250, 253, 261, 265, 271, 276, 277, 285]
output_dir = "pds_img"
os.makedirs(output_dir, exist_ok=True)
sampling=100
keywords = ["BIEQI", "BINQI", "BITQI"] #Falta cambiar de I a H
path= "C:/Users/ameli/Desktop/Cassini/pds_img"

In [3]:
#lista de angulos
def procesar_angulos(carpeta,sampling):
    img_dict = { "inc": None}

    for archivo in os.listdir(carpeta):
        if archivo.startswith("BIE"):
            img_dict["inc"] = PDS3Image.open(os.path.join(carpeta, archivo))

    inc = img_dict["inc"].image[::sampling, ::sampling].flatten()
    listInc=[]
    for i in range(inc.shape[0]):
        listInc.append(inc[i])
    
    return listInc

In [4]:
def generar_hull(carpeta, sampling=100, alpha=0.1):

    img_dict = {"lat": None, "lon": None, "inc": None}

    for archivo in os.listdir(carpeta):
        if archivo.startswith("BIN"):
            img_dict["lat"] = PDS3Image.open(os.path.join(carpeta, archivo))
        elif archivo.startswith("BIT"):
            img_dict["lon"] = PDS3Image.open(os.path.join(carpeta, archivo))
        elif archivo.startswith("BIE"):
            img_dict["inc"] = PDS3Image.open(os.path.join(carpeta, archivo))

    inc = img_dict["inc"].image[::sampling, ::sampling].flatten()
    lat = img_dict["lat"].image[::sampling, ::sampling].flatten()
    lon = img_dict["lon"].image[::sampling, ::sampling].flatten()

    lat = lat[inc > 0]
    lon = lon[inc > 0]

    latLonList = np.zeros((lon.shape[0], 2))
    for i in range(lat.shape[0]):
        latLonList[i] = np.array([lat[i], lon[i]])
    # Crear la lista de coordenadas (lat, lon)
    latLonList = np.zeros((lon.shape[0], 2))
    for i in range(lat.shape[0]):
        latLonList[i] = np.asarray([lat[i], lon[i]])

    # Generar el concave hull (alphashape)
    concave_hull = alphashape.alphashape(latLonList, alpha)

    if isinstance(concave_hull, Polygon):
        # Obtener los puntos del contorno exterior
        exterior_coords = list(concave_hull.exterior.coords)
    
    return exterior_coords


In [5]:
def descargar_y_extraer_zip(url, destino):
    try:
        print(f"Descargando {url} ...")
        respuesta = requests.get(url)
        
        if respuesta.status_code != 200:
            print(f"Error al descargar {url} (status: {respuesta.status_code}).")
            return

        # Leer el contenido del ZIP en memoria y extraerlo en `destino`
        with zipfile.ZipFile(io.BytesIO(respuesta.content)) as z:
            z.extractall(destino)
            print(f"Archivos extraídos en {destino}")

    except Exception as e:
        print(f"Error inesperado al procesar {url}: {e}")



In [6]:
def explorar_volumen(volumen_actual, output_dir, keywords):
    if volumen_actual < 100:
        base_url = f"https://planetarydata.jpl.nasa.gov/img/data/cassini/cassini_orbiter/CORADR_00{volumen_actual}/DATA/BIDR/"
    else:
        base_url = f"https://planetarydata.jpl.nasa.gov/img/data/cassini/cassini_orbiter/CORADR_0{volumen_actual}/DATA/BIDR/"

    volumen_dir = os.path.join(output_dir, f"volumen_{volumen_actual}")
    os.makedirs(volumen_dir, exist_ok=True)
    print(f"Carpeta creada para el volumen {volumen_actual}: {volumen_dir}")
    
    try:
        respuesta = requests.get(base_url)
        if respuesta.status_code != 200:
            print(f"No se pudo acceder al volumen {volumen_actual}. (status: {respuesta.status_code})")
            return volumen_dir

        soup = BeautifulSoup(respuesta.content, 'html.parser')
        enlaces = soup.find_all('a', href=True)

        for keyword in keywords:
            zip_encontrado = False
            for enlace in enlaces:
                href = enlace['href']
                if href.endswith('.ZIP') and keyword in href:
                    url_zip = base_url + href
                    descargar_y_extraer_zip(url_zip, volumen_dir)
                    zip_encontrado = True
                    print(f"Archivo '{href}' encontrado y descargado para la palabra clave '{keyword}'.")
                    break

            if not zip_encontrado:
                print(f"No se encontró el archivo ZIP con la palabra clave '{keyword}' en el volumen {volumen_actual}.")

        return volumen_dir

    except requests.exceptions.RequestException as e:
        print(f"Ocurrió un error al acceder a {base_url}: {e}")
        return volumen_dir
    except Exception as e:
        print(f"Ocurrió un error al explorar {base_url}: {e}")
        return volumen_dir


In [7]:
def procesar_volumenes(volumenes, output_dir, sampling=100):
    listIndices = []
    for volumen in volumenes:
        #carpeta_destino = explorar_volumen(volumen,output_dir,keywords) #comentar esta linea de codigo si ya tenes los archivos descargados
        carpeta_destino=os.path.join(output_dir, f"volumen_{volumen}")
        # Verificar si la carpeta contiene archivos después de la descarga
        if carpeta_destino and os.path.isdir(carpeta_destino):
            print(f"Archivos en {carpeta_destino}: {os.listdir(carpeta_destino)}")
            # Aquí podrías continuar con el procesamiento de las imágenes en la carpeta
            try:
                #listInc = promedio_angulos(procesar_angulos(carpeta_destino,sampling))
                latLonList = generar_hull(carpeta_destino, sampling=100)  # 'sampling' ajusta el nivel de detalle
                print(f"Puntos de latitud y longitud extraídos: {latLonList[:5]}...")  # Mostrar los primeros puntos
                if latLonList is not None and len(latLonList) > 0:  
                    listIndices.append((volumen,latLonList))
                    print(f"Contorno guardado para el volumen {volumen}")
                else:
                    print(f"No se pudo generar contorno para el volumen {volumen}")
            except Exception as e:
                print(f"Error al procesar las imágenes en {carpeta_destino}: {e}")
                latLonList = []
            else:
                print(f"No se encontraron puntos en el volumen {volumen}")
            
        else:
            print(f"No se pudo explorar o descargar archivos para el volumen {volumen}")

    # Mostrar el resultado final
    return listIndices


In [8]:

def interseccion(s, precision=5):
    intersecting_pairs = []
    for i in range(len(s)):
        j = i + 1
        # Redondear y validar el primer polígono
        poly1_coords = [(x,y) for x, y in s[i][1]]
        poly1 = Polygon(poly1_coords)
        if not poly1.is_valid:
            poly1 = poly1.buffer(0)  # Intentar corregir

        while j < len(s):
            # Redondear y validar el segundo polígono
            poly2_coords = [(x, y) for x, y in s[j][1]]
            poly2 = Polygon(poly2_coords)
            if not poly2.is_valid:
                poly2 = poly2.buffer(0)  # Intentar corregir
            
            try:
                if s[i][0] != s[j][0] and poly1.intersects(poly2):
                    # Aplicar buffer de 0 al momento de la intersección
                    area_interseccion = poly1.intersection(poly2)
                    if area_interseccion.geom_type == "MultiPolygon":
                        intersecting_pairs.append(((s[i][0], s[j][0]),"tiene dos áreas de intersección"))
                    else:
                        intersecting_pairs.append(((s[i][0], s[j][0]),area_interseccion))
            except TopologicalError as e:
                print(f"Error de topología entre los polígonos {s[i][0]} y {s[j][0]}: {e}")
            
            j += 1 
    return intersecting_pairs

listIndices=procesar_volumenes(volumenes,output_dir,sampling)
print(interseccion(listIndices,output_dir))







Archivos en pds_img\volumen_35: ['BIEQI49N071_D035_T00AS01_V03.IMG', 'BINQI49N071_D035_T00AS01_V03.IMG', 'BITQI49N071_D035_T00AS01_V03.IMG']
Puntos de latitud y longitud extraídos: [(46.283935546875, 51.99863052368164), (46.895751953125, 52.07122039794922), (47.50933837890625, 52.14112854003906), (48.12469482421875, 52.20833969116211), (48.741729736328125, 52.2728385925293)]...
Contorno guardado para el volumen 35
No se encontraron puntos en el volumen 35
Archivos en pds_img\volumen_45: ['BIEQI22N068_D045_T003S01_V03.IMG', 'BINQI22N068_D045_T003S01_V03.IMG', 'BITQI22N068_D045_T003S01_V03.IMG']
Puntos de latitud y longitud extraídos: [(89.21600341796875, 15.183061599731445), (86.58013916015625, 16.10784339904785), (86.1895751953125, 16.180526733398438), (85.79864501953125, 16.25196647644043), (85.40728759765625, 16.32215690612793)]...
Contorno guardado para el volumen 45
No se encontraron puntos en el volumen 45
Archivos en pds_img\volumen_65: ['BIEQI10S251_D065_T008S01_V03.IMG', 'BINQI

In [9]:
def procesar_incidencia(carpeta, sampling=100):
    img_dict = {"lat": None, "lon": None, "inc": None}
    
    # Leer archivos en la carpeta
    for archivo in os.listdir(carpeta):
        if archivo.startswith("BIE"):  # Ajustar según el prefijo del archivo
            img_dict["inc"] = PDS3Image.open(os.path.join(carpeta, archivo))
    
    # Validar que se encontró el archivo de incidencia
    if img_dict["inc"] is None:
        raise FileNotFoundError("No se encontró el archivo de ángulos de incidencia en la carpeta.")
    
    # Extraer datos de incidencia
    inc = img_dict["inc"].image[::sampling, ::sampling].flatten()  # Reducir datos con sampling
    inc_validos = inc[inc > 0]  # Filtrar valores inválidos como ceros o negativos
    
    if len(inc_validos) == 0:
        raise ValueError("No hay valores válidos de incidencia en los datos.")
    
    # Calcular el promedio de ángulos de incidencia válidos
    promedio = np.mean(inc_validos)
    print(f"Promedio de ángulo de incidencia: {promedio:.2f} grados")
    return promedio

# Uso de la función



In [10]:
def limpiar_lista(intersecting_pairs, output_dir, sampling=100):
    nueva_lista = []
    for tupla,area in intersecting_pairs:
        area_interseccion=area
        carpeta_1 = f"{output_dir}/volumen_{tupla[0]}"
        carpeta_2 = f"{output_dir}/volumen_{tupla[1]}"
        inc_1 = procesar_incidencia(carpeta_1, sampling)
        inc_2 = procesar_incidencia(carpeta_2, sampling)
        resta = abs(inc_1 - inc_2)
        if resta >= 20:
            nueva_lista.append((tupla,inc_1,inc_2, resta,area_interseccion))
    
    # Guardar la lista final en un archivo de texto
    archivo_salida = os.path.join(output_dir, "lista_intersecciones.txt")
    with open(archivo_salida, "w") as f:
        for elemento in nueva_lista:
            f.write(f"{elemento}\n")
    
    print(f"Lista guardada en {archivo_salida}")
    return nueva_lista

# Uso de la función
lista_intersecciones = interseccion(listIndices, output_dir)
resultado_final = limpiar_lista(lista_intersecciones, output_dir)


Promedio de ángulo de incidencia: 24.40 grados
Promedio de ángulo de incidencia: 21.65 grados
Promedio de ángulo de incidencia: 24.40 grados
Promedio de ángulo de incidencia: 27.08 grados
Promedio de ángulo de incidencia: 24.40 grados
Promedio de ángulo de incidencia: 21.66 grados
Promedio de ángulo de incidencia: 24.40 grados
Promedio de ángulo de incidencia: 35.31 grados
Promedio de ángulo de incidencia: 19.09 grados
Promedio de ángulo de incidencia: 21.57 grados
Promedio de ángulo de incidencia: 19.09 grados
Promedio de ángulo de incidencia: 14.80 grados
Promedio de ángulo de incidencia: 19.09 grados
Promedio de ángulo de incidencia: 21.66 grados
Promedio de ángulo de incidencia: 21.00 grados
Promedio de ángulo de incidencia: 26.51 grados
Promedio de ángulo de incidencia: 21.00 grados
Promedio de ángulo de incidencia: 13.64 grados
Promedio de ángulo de incidencia: 21.00 grados
Promedio de ángulo de incidencia: 22.17 grados
Promedio de ángulo de incidencia: 21.00 grados
Promedio de á